In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
# download the cifar10 dataset into the data directory
dataset, info = tfds.load('cifar10', with_info=True, as_supervised=True,data_dir='data')

In [3]:
#split into train and test datasets
train_dataset, test_dataset = dataset['train'], dataset['test']

In [4]:
#create a function to normalize and resize the images
def normalize_and_resize(image, label):
    image=tf.cast(image, tf.float32)
    image=tf.divide(image, 255)
    image=tf.image.resize(image, [28,28])
    return image, label

#create a function to augment the images
def augment(image, label):
    image=tf.image.random_flip_left_right(image)
    image=tf.image.random_flip_up_down(image)
    image=tf.image.random_brightness(image, max_delta=0.5)
    image=tf.image.random_contrast(image, lower=0.2, upper=1.8)
    image=tf.image.random_hue(image, max_delta=0.2)
    image=tf.image.random_saturation(image, lower=0.2, upper=1.8)
    return image, label

In [5]:
#modify the train and test datasets using the function
train_dataset=train_dataset.map(normalize_and_resize).cache().map(augment).shuffle(1000).batch(64).repeat()
test_dataset=test_dataset.map(normalize_and_resize).cache().batch(64)

In [6]:
#create a basic CNN model using Tensorflow
basic_model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

basic_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
#create an early stopping callback
early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)

#create a lr plateau callback
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)

In [ ]:
#fit the model with the data and the callbacks
history=basic_model.fit(train_dataset, epochs=32, steps_per_epoch=60000//64, validation_data=test_dataset, callbacks=[early_stop, reduce_lr],verbose=2)

Error: Session cannot generate requests